In [157]:
from acquire import get_iris_data
from prepare import prep_iris
from acquire import get_titanic_data
from prepare import prep_titanic_data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def get_confusion_metrics(cnf):
    FP = cnf.sum(axis=0) - np.diag(cnf)  
    FN = (cnf.sum(axis=1)) - np.diag(cnf)
    TP = np.diag(cnf)
    TN = cnf.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    print(f'Recall: {TPR}')
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
    print(f'True Negative Rate: {TNR}')
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    print(f'Precision: {PPV}')
    # Negative predictive value
    NPV = TN/(TN+FN)
    print(f'Negative Predictive Value: {NPV}')
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    print(f'False positive Rate: {FPR}')
    # False negative rate
    FNR = FN/(TP+FN)
    print(f'False Negative Rate: {FNR}')
    # False discovery rate
    FDR = FP/(TP+FP)
    print(f'False Discovery Rate: {FDR}')
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    print(f'Overall Accuracy: {ACC}')

In [158]:
df = get_titanic_data()

In [159]:
df = prep_titanic_data(df)
df.dropna(inplace=True)

In [160]:
train, test = train_test_split(df)

In [161]:
scaler = MinMaxScaler()

scaler.fit(train[['age', 'fare']])

train[['age', 'fare']] = scaler.transform(train[['age', 'fare']])
test[['age', 'fare']] = scaler.transform(test[['age', 'fare']])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyda

In [162]:
logit = LogisticRegression()

logit.fit(train[['pclass', 'age', 'fare', 'sibsp', 'parch']], train.survived)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [163]:
logit.predict_proba(train[['pclass', 'age', 'fare', 'sibsp', 'parch']])

array([[0.73376812, 0.26623188],
       [0.63399913, 0.36600087],
       [0.79558476, 0.20441524],
       ...,
       [0.60809021, 0.39190979],
       [0.72701758, 0.27298242],
       [0.60750745, 0.39249255]])

In [164]:
train['prediction'] = logit.predict(train[['pclass', 'age', 'fare', 'sibsp', 'parch']])

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [165]:
logit.score(train[['pclass', 'age', 'fare', 'sibsp', 'parch']], train.survived)

0.7102803738317757

In [166]:
print(classification_report(train.survived, train.prediction))

confusion_matrix(train.survived, train.prediction)

              precision    recall  f1-score   support

           0       0.72      0.86      0.78       322
           1       0.70      0.48      0.57       213

   micro avg       0.71      0.71      0.71       535
   macro avg       0.71      0.67      0.68       535
weighted avg       0.71      0.71      0.70       535



array([[277,  45],
       [110, 103]])

In [167]:
titanic_matrix = confusion_matrix(train.survived, train.prediction)
print(f'True Negative: {titanic_matrix[0][0]}')
print(f'False Positive: {titanic_matrix[0][1]}')
print(f'False Negative: {titanic_matrix[1][0]}')
print(f'True Positive: {titanic_matrix[1][1]}')
true_pos = titanic_matrix[1][1]
true_neg = titanic_matrix[0][0]
false_pos = titanic_matrix[0][1]
false_neg = titanic_matrix[1][0]

True Negative: 277
False Positive: 45
False Negative: 110
True Positive: 103


In [168]:
total_obs = titanic_matrix[0][0] + titanic_matrix[0][1] + titanic_matrix[1][0] + titanic_matrix[1][1]
accuracy = (true_pos + true_neg) / total_obs
print(f'Accuracy: {accuracy}')
precision = true_pos / (true_pos + false_pos)
print(f'Precision: {precision}')
recall = true_pos / (true_pos + false_neg)
print(f'Recall (true positive rate): {recall}')
f1 = recall + precision / 2
print(f'f1: {f1}')
fp_rate = false_pos / (false_pos + true_neg)
print(f'False Positive Rate (specificity): {fp_rate}')
tn_rate = true_neg / (true_neg + false_pos)
print(f'True Negative Rate: {tn_rate}')
fn_rate = false_neg / (true_pos + false_neg)
print('False Negative Rate: {:2}'.format(fn_rate))
print(f'support: {total_obs}')


Accuracy: 0.7102803738317757
Precision: 0.6959459459459459
Recall (true positive rate): 0.4835680751173709
f1: 0.8315410480903438
False Positive Rate (specificity): 0.13975155279503104
True Negative Rate: 0.860248447204969
False Negative Rate: 0.5164319248826291
support: 535


In [169]:
import matplotlib.pyplot as plt
# y_pred_proba = [i for i in train['prediction']]
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.scatter(y_pred_proba, train['prediction'])

## Let's run through the model building process with another 'solver' parameter.  Our best option should be the liblinear option (which is the default) as we are working with a small data set and not a multiclass problem

In [170]:
X = df[['pclass','age','fare','sibsp','parch']]
y = df[['survived']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

X_train.head()

,pclass,age,fare,sibsp,parch
60,3,22.0,7.2292,0,0
348,3,3.0,15.9000,1,1
606,3,30.0,7.8958,0,0
195,1,58.0,146.5208,0,0
56,2,21.0,10.5000,0,0


In [171]:
logit = LogisticRegression(C=1, class_weight={1:2}, random_state = 123, solver='saga')
logit.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, class_weight={1: 2}, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [172]:
y_pred = logit.predict(X_train)
y_pred_proba = logit.predict_proba(X_train)

In [173]:
logit.score(X_train, y_train)

0.6472945891783567

In [174]:
# saga performed at 0.64 accuracy, 
#less than our first model on default liblinear

In [175]:
dfi = get_iris_data()

In [176]:
dfi = prep_iris(dfi)

In [177]:
dfi.head()

,species,sepal_length,sepal_width,petal_length,petal_width,species_encode
0,setosa,5.1,3.5,1.4,0.2,0
1,setosa,4.9,3.0,1.4,0.2,0
2,setosa,4.7,3.2,1.3,0.2,0
3,setosa,4.6,3.1,1.5,0.2,0
4,setosa,5.0,3.6,1.4,0.2,0


In [178]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

X = dfi.drop(columns=['species', 'species_encode'])
y = dfi[['species']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 123)
X_train.head()

,sepal_length,sepal_width,petal_length,petal_width
114,5.8,2.8,5.1,2.4
136,6.3,3.4,5.6,2.4
53,5.5,2.3,4.0,1.3
19,5.1,3.8,1.5,0.3
38,4.4,3.0,1.3,0.2


In [179]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)


In [180]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [181]:
y_pred = clf.predict(X_train)
y_pred[0:4]

array(['virginica', 'virginica', 'versicolor', 'setosa'], dtype=object)

In [182]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [0.   , 0.   , 1.   ],
       [1.   , 0.   , 0.   ],
       [0.   , 0.   , 1.   ],
       [0.   , 0.975, 0.025],
       [0.   , 0.975, 0.025],
       [1.   , 0.   , 0.   ],
       [0.

In [183]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.98


In [184]:
confusion_matrix(y_train, y_pred)

array([[32,  0,  0],
       [ 0, 40,  0],
       [ 0,  2, 31]])

In [185]:
sorted(y_train.species.unique())

['setosa', 'versicolor', 'virginica']

In [186]:
y_train.species.value_counts()

versicolor    40
virginica     33
setosa        32
Name: species, dtype: int64

In [187]:
import pandas as pd

labels = sorted(y_train.species.unique())

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,setosa,versicolor,virginica
setosa,32,0,0
versicolor,0,40,0
virginica,0,2,31


In [188]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.95      1.00      0.98        40
   virginica       1.00      0.94      0.97        33

   micro avg       0.98      0.98      0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105



In [189]:
import graphviz

from graphviz import Graph

dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.render('iris_decision_tree', view=True)

'iris_decision_tree.pdf'

In [190]:
import numpy as np
import pandas as pd
tree_cnf_matrix = confusion_matrix(y_train, y_pred)
print(tree_cnf_matrix)
print(np.diag(tree_cnf_matrix))
tree_cnf_matrix.sum(axis=1)

[[32  0  0]
 [ 0 40  0]
 [ 0  2 31]]
[32 40 31]


array([32, 40, 33])

In [191]:

FP = tree_cnf_matrix.sum(axis=0) - np.diag(tree_cnf_matrix)  
FN = (tree_cnf_matrix.sum(axis=1)) - np.diag(tree_cnf_matrix)
TP = np.diag(tree_cnf_matrix)
TN = tree_cnf_matrix.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print(f'Recall: {TPR}')
# Specificity or true negative rate
TNR = TN/(TN+FP) 
print(f'True Negative Rate: {TNR}')
# Precision or positive predictive value
PPV = TP/(TP+FP)
print(f'Precision: {PPV}')
# Negative predictive value
NPV = TN/(TN+FN)
print(f'Negative Predictive Value: {NPV}')
# Fall out or false positive rate
FPR = FP/(FP+TN)
print(f'False positive Rate: {FPR}')
# False negative rate
FNR = FN/(TP+FN)
print(f'False Negative Rate: {FNR}')
# False discovery rate
FDR = FP/(TP+FP)
print(f'False Discovery Rate: {FDR}')
# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(f'Overall Accuracy: {ACC}')

Recall: [1.         1.         0.93939394]
True Negative Rate: [1.         0.96923077 1.        ]
Precision: [1.         0.95238095 1.        ]
Negative Predictive Value: [1.         1.         0.97297297]
False positive Rate: [0.         0.03076923 0.        ]
False Negative Rate: [0.         0.         0.06060606]
False Discovery Rate: [0.         0.04761905 0.        ]
Overall Accuracy: [1.         0.98095238 0.98095238]


## Decision Tree (Titanic)

In [192]:
X_t = train[['pclass','age','fare','sibsp','parch']]
y_t = train[['survived']]
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_t, y_t, test_size = 0.30, random_state = 123)

In [193]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)

In [194]:
clf.fit(X_train_t, y_train_t)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [196]:
y_pred = clf.predict(X_train_t)
y_pred[0:4]

array([0, 0, 0, 0])

In [199]:
y_pred_proba = clf.predict_proba(X_train_t)
y_pred_proba[0:10]

array([[0.91304348, 0.08695652],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.28169014, 0.71830986],
       [0.28169014, 0.71830986],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.85714286, 0.14285714]])

In [200]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train_t, y_train_t)))

Accuracy of Decision Tree classifier on training set: 0.79


In [202]:
confusion_matrix(y_train_t, y_pred)

array([[180,  41],
       [ 39, 114]])

In [204]:
print(classification_report(y_train_t, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.81      0.82       221
           1       0.74      0.75      0.74       153

   micro avg       0.79      0.79      0.79       374
   macro avg       0.78      0.78      0.78       374
weighted avg       0.79      0.79      0.79       374



In [205]:
get_confusion_metrics(confusion_matrix(y_train_t, y_pred))

Recall: [0.81447964 0.74509804]
True Negative Rate: [0.74509804 0.81447964]
Precision: [0.82191781 0.73548387]
Negative Predictive Value: [0.73548387 0.82191781]
False positive Rate: [0.25490196 0.18552036]
False Negative Rate: [0.18552036 0.25490196]
False Discovery Rate: [0.17808219 0.26451613]
Overall Accuracy: [0.78609626 0.78609626]


In [206]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)

In [207]:
clf.fit(X_train_t, y_train_t)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [208]:
y_pred = clf.predict(X_train_t)
y_pred[0:4]

array([0, 0, 0, 0])

In [209]:
y_pred_proba = clf.predict_proba(X_train_t)
y_pred_proba[0:10]

array([[0.88      , 0.12      ],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.28169014, 0.71830986],
       [0.28169014, 0.71830986],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.78431373, 0.21568627],
       [0.85714286, 0.14285714]])

In [210]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train_t, y_train_t)))

Accuracy of Decision Tree classifier on training set: 0.79


In [211]:
confusion_matrix(y_train_t, y_pred)

array([[181,  40],
       [ 40, 113]])

In [212]:
print(classification_report(y_train_t, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       221
           1       0.74      0.74      0.74       153

   micro avg       0.79      0.79      0.79       374
   macro avg       0.78      0.78      0.78       374
weighted avg       0.79      0.79      0.79       374



In [213]:
get_confusion_metrics(confusion_matrix(y_train_t, y_pred))

Recall: [0.81900452 0.73856209]
True Negative Rate: [0.73856209 0.81900452]
Precision: [0.81900452 0.73856209]
Negative Predictive Value: [0.73856209 0.81900452]
False positive Rate: [0.26143791 0.18099548]
False Negative Rate: [0.18099548 0.26143791]
False Discovery Rate: [0.18099548 0.26143791]
Overall Accuracy: [0.78609626 0.78609626]


## KNN (titanic)

In [48]:
from sklearn.neighbors import KNeighborsClassifier


In [78]:
X_train_t.shape

(374, 5)

In [79]:
X_test_t.shape

(161, 5)

In [80]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train_t, y_train_t)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [81]:
y_pred = knn.predict(X_train_t)

y_pred_proba = knn.predict_proba(X_train_t)

print("accuracy of KNN on training set: {:.2f}".format(knn.score(X_train_t, y_train_t)))

accuracy of KNN on training set: 0.78


In [82]:
print(confusion_matrix(y_train_t, y_pred))

print(classification_report(y_train_t, y_pred))

[[208  28]
 [ 54  84]]
              precision    recall  f1-score   support

           0       0.79      0.88      0.84       236
           1       0.75      0.61      0.67       138

   micro avg       0.78      0.78      0.78       374
   macro avg       0.77      0.75      0.75       374
weighted avg       0.78      0.78      0.78       374



In [83]:
ttnc_cnf = confusion_matrix(y_train_t, y_pred)

get_confusion_metrics(ttnc_cnf)

Recall: [0.88135593 0.60869565]
True Negative Rate: [0.60869565 0.88135593]
Precision: [0.79389313 0.75      ]
Negative Predictive Value: [0.75       0.79389313]
False positive Rate: [0.39130435 0.11864407]
False Negative Rate: [0.11864407 0.39130435]
False Discovery Rate: [0.20610687 0.25      ]
Overall Accuracy: [0.78074866 0.78074866]


## set k to 10:

In [84]:
knn = KNeighborsClassifier(n_neighbors=10, weights='uniform')
knn.fit(X_train_t, y_train_t)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [85]:
y_pred = knn.predict(X_train_t)

y_pred_proba = knn.predict_proba(X_train_t)

print("accuracy of KNN on training set: {:.2f}".format(knn.score(X_train_t, y_train_t)))

accuracy of KNN on training set: 0.75


In [86]:
print(confusion_matrix(y_train_t, y_pred))

print(classification_report(y_train_t, y_pred))

[[215  21]
 [ 71  67]]
              precision    recall  f1-score   support

           0       0.75      0.91      0.82       236
           1       0.76      0.49      0.59       138

   micro avg       0.75      0.75      0.75       374
   macro avg       0.76      0.70      0.71       374
weighted avg       0.76      0.75      0.74       374



In [87]:
get_confusion_metrics(confusion_matrix(y_train_t, y_pred))

Recall: [0.91101695 0.48550725]
True Negative Rate: [0.48550725 0.91101695]
Precision: [0.75174825 0.76136364]
Negative Predictive Value: [0.76136364 0.75174825]
False positive Rate: [0.51449275 0.08898305]
False Negative Rate: [0.08898305 0.51449275]
False Discovery Rate: [0.24825175 0.23863636]
Overall Accuracy: [0.7540107 0.7540107]


## Set k to 20:

In [88]:
knn = KNeighborsClassifier(n_neighbors=20, weights='uniform')
knn.fit(X_train_t, y_train_t)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='uniform')

In [89]:
y_pred = knn.predict(X_train_t)

y_pred_proba = knn.predict_proba(X_train_t)

print("accuracy of KNN on training set: {:.2f}".format(knn.score(X_train_t, y_train_t)))

accuracy of KNN on training set: 0.74


In [90]:
print(confusion_matrix(y_train_t, y_pred))

print(classification_report(y_train_t, y_pred))

[[207  29]
 [ 70  68]]
              precision    recall  f1-score   support

           0       0.75      0.88      0.81       236
           1       0.70      0.49      0.58       138

   micro avg       0.74      0.74      0.74       374
   macro avg       0.72      0.68      0.69       374
weighted avg       0.73      0.74      0.72       374



In [91]:
get_confusion_metrics(confusion_matrix(y_train_t, y_pred))

Recall: [0.87711864 0.49275362]
True Negative Rate: [0.49275362 0.87711864]
Precision: [0.74729242 0.70103093]
Negative Predictive Value: [0.70103093 0.74729242]
False positive Rate: [0.50724638 0.12288136]
False Negative Rate: [0.12288136 0.50724638]
False Discovery Rate: [0.25270758 0.29896907]
Overall Accuracy: [0.73529412 0.73529412]


### our k=5 model perfomed the best across the board.  Accuracy and precision topped in this model.  This makes sense with our understanding of the k-nearest neighbors modeling system, as we will cease to gain accuracy at a threshhold of neighbors as we increase our k.

In [92]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn_fit = knn.fit(X_train_t, y_train_t)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


## KNN (iris)

In [113]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [114]:
knn.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [115]:
y_pred = knn.predict(X_train)

In [118]:
y_pred_proba = knn.predict_proba(X_train)
y_pred_proba[0:10]

array([[0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0.2, 0.8],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 1. , 0. ]])

In [119]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.98


In [120]:
print(confusion_matrix(y_train, y_pred))

[[32  0  0]
 [ 0 39  1]
 [ 0  1 32]]


In [121]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.97      0.97      0.97        40
   virginica       0.97      0.97      0.97        33

   micro avg       0.98      0.98      0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105



In [123]:
get_confusion_metrics(confusion_matrix(y_train, y_pred))

Recall: [1.         0.975      0.96969697]
True Negative Rate: [1.         0.98461538 0.98611111]
Precision: [1.         0.975      0.96969697]
Negative Predictive Value: [1.         0.98461538 0.98611111]
False positive Rate: [0.         0.01538462 0.01388889]
False Negative Rate: [0.         0.025      0.03030303]
False Discovery Rate: [0.         0.025      0.03030303]
Overall Accuracy: [1.         0.98095238 0.98095238]


## k=10 (iris)

In [124]:
knn = KNeighborsClassifier(n_neighbors=10, weights='uniform')

In [125]:
knn.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [126]:
y_pred = knn.predict(X_train)

In [127]:
y_pred_proba = knn.predict_proba(X_train)


In [128]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.97


In [129]:
print(confusion_matrix(y_train, y_pred))

[[32  0  0]
 [ 0 39  1]
 [ 0  2 31]]


In [130]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.95      0.97      0.96        40
   virginica       0.97      0.94      0.95        33

   micro avg       0.97      0.97      0.97       105
   macro avg       0.97      0.97      0.97       105
weighted avg       0.97      0.97      0.97       105



In [131]:
get_confusion_metrics(confusion_matrix(y_train, y_pred))

Recall: [1.         0.975      0.93939394]
True Negative Rate: [1.         0.96923077 0.98611111]
Precision: [1.         0.95121951 0.96875   ]
Negative Predictive Value: [1.         0.984375   0.97260274]
False positive Rate: [0.         0.03076923 0.01388889]
False Negative Rate: [0.         0.025      0.06060606]
False Discovery Rate: [0.         0.04878049 0.03125   ]
Overall Accuracy: [1.         0.97142857 0.97142857]


## k=20 (iris)

In [132]:
knn = KNeighborsClassifier(n_neighbors=20, weights='uniform')

In [133]:
knn.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=20, p=2,
           weights='uniform')

In [134]:
y_pred = knn.predict(X_train)

In [135]:
y_pred_proba = knn.predict_proba(X_train)


In [136]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.96


In [137]:
print(confusion_matrix(y_train, y_pred))

[[32  0  0]
 [ 0 39  1]
 [ 0  3 30]]


In [138]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.93      0.97      0.95        40
   virginica       0.97      0.91      0.94        33

   micro avg       0.96      0.96      0.96       105
   macro avg       0.97      0.96      0.96       105
weighted avg       0.96      0.96      0.96       105



In [139]:
get_confusion_metrics(confusion_matrix(y_train, y_pred))

Recall: [1.         0.975      0.90909091]
True Negative Rate: [1.         0.95384615 0.98611111]
Precision: [1.         0.92857143 0.96774194]
Negative Predictive Value: [1.         0.98412698 0.95945946]
False positive Rate: [0.         0.04615385 0.01388889]
False Negative Rate: [0.         0.025      0.09090909]
False Discovery Rate: [0.         0.07142857 0.03225806]
Overall Accuracy: [1.         0.96190476 0.96190476]


### k=5 also performs the best on the iris dataset.

In [142]:
knn_iris = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn_fit = knn.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


## Random Forest

In [93]:
from sklearn.ensemble import RandomForestClassifier

In [95]:
rf = RandomForestClassifier(min_samples_leaf = 1, max_depth = 20, random_state =123)

In [96]:
rf.fit(X_train_t, y_train_t)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [97]:
print(rf.feature_importances_)

[0.056422   0.40731214 0.41255481 0.06379346 0.05991758]


In [101]:
y_pred_proba = rf.predict_proba(X_train_t)
y_pred_proba[0:10]

array([[0. , 1. ],
       [0. , 1. ],
       [1. , 0. ],
       [0.2, 0.8],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [0.9, 0.1],
       [0.9, 0.1],
       [0.2, 0.8]])

In [102]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train_t, y_train_t)))

Accuracy of random forest classifier on training set: 0.97


In [103]:
print(confusion_matrix(y_train_t, y_pred))


[[207  29]
 [ 70  68]]


In [105]:
print(classification_report(y_train_t, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.88      0.81       236
           1       0.70      0.49      0.58       138

   micro avg       0.74      0.74      0.74       374
   macro avg       0.72      0.68      0.69       374
weighted avg       0.73      0.74      0.72       374



In [106]:
get_confusion_metrics(confusion_matrix(y_train_t, y_pred))

Recall: [0.87711864 0.49275362]
True Negative Rate: [0.49275362 0.87711864]
Precision: [0.74729242 0.70103093]
Negative Predictive Value: [0.70103093 0.74729242]
False positive Rate: [0.50724638 0.12288136]
False Negative Rate: [0.12288136 0.50724638]
False Discovery Rate: [0.25270758 0.29896907]
Overall Accuracy: [0.73529412 0.73529412]


In [107]:
rf = RandomForestClassifier(min_samples_leaf = 5, max_depth = 3, random_state =123)

In [108]:
rf.fit(X_train_t, y_train_t)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [109]:
print(rf.feature_importances_)

[0.16475373 0.28676699 0.48000637 0.03485253 0.03362037]


In [110]:
y_pred_proba = rf.predict_proba(X_train_t)
y_pred_proba[0:10]

array([[0.38860258, 0.61139742],
       [0.26872774, 0.73127226],
       [0.77703641, 0.22296359],
       [0.42098217, 0.57901783],
       [0.41253178, 0.58746822],
       [0.77703641, 0.22296359],
       [0.73166577, 0.26833423],
       [0.55450795, 0.44549205],
       [0.71442685, 0.28557315],
       [0.40837925, 0.59162075]])

In [111]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train_t, y_train_t)))

Accuracy of random forest classifier on training set: 0.74


In [112]:
get_confusion_metrics(confusion_matrix(y_train_t, y_pred))

Recall: [0.87711864 0.49275362]
True Negative Rate: [0.49275362 0.87711864]
Precision: [0.74729242 0.70103093]
Negative Predictive Value: [0.70103093 0.74729242]
False positive Rate: [0.50724638 0.12288136]
False Negative Rate: [0.12288136 0.50724638]
False Discovery Rate: [0.25270758 0.29896907]
Overall Accuracy: [0.73529412 0.73529412]


## Random Forest (iris)

In [143]:
rf = RandomForestClassifier(min_samples_leaf = 1, max_depth = 20, random_state =123)

In [144]:
rf.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [145]:
y_pred_proba = rf.predict_proba(X_train)
y_pred_proba[0:10]

array([[0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0.1, 0.9],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0.9, 0.1]])

In [147]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 1.00


In [148]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.93      0.97      0.95        40
   virginica       0.97      0.91      0.94        33

   micro avg       0.96      0.96      0.96       105
   macro avg       0.97      0.96      0.96       105
weighted avg       0.96      0.96      0.96       105



In [149]:
get_confusion_metrics(confusion_matrix(y_train, y_pred))

Recall: [1.         0.975      0.90909091]
True Negative Rate: [1.         0.95384615 0.98611111]
Precision: [1.         0.92857143 0.96774194]
Negative Predictive Value: [1.         0.98412698 0.95945946]
False positive Rate: [0.         0.04615385 0.01388889]
False Negative Rate: [0.         0.025      0.09090909]
False Discovery Rate: [0.         0.07142857 0.03225806]
Overall Accuracy: [1.         0.96190476 0.96190476]


In [151]:
rf = RandomForestClassifier(min_samples_leaf = 5, max_depth = 3, random_state =123)

In [152]:
rf.fit(X_train, y_train)

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [153]:
y_pred_proba = rf.predict_proba(X_train)
y_pred_proba[0:10]

array([[0.        , 0.04761905, 0.95238095],
       [0.        , 0.00769231, 0.99230769],
       [0.        , 0.96401822, 0.03598178],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.04102564, 0.95897436],
       [0.92857143, 0.02857143, 0.04285714],
       [1.        , 0.        , 0.        ],
       [0.        , 0.95793651, 0.04206349],
       [0.        , 0.96507937, 0.03492063]])

In [154]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.96


In [155]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        32
  versicolor       0.93      0.97      0.95        40
   virginica       0.97      0.91      0.94        33

   micro avg       0.96      0.96      0.96       105
   macro avg       0.97      0.96      0.96       105
weighted avg       0.96      0.96      0.96       105



In [156]:
get_confusion_metrics(confusion_matrix(y_train, y_pred))

Recall: [1.         0.975      0.90909091]
True Negative Rate: [1.         0.95384615 0.98611111]
Precision: [1.         0.92857143 0.96774194]
Negative Predictive Value: [1.         0.98412698 0.95945946]
False positive Rate: [0.         0.04615385 0.01388889]
False Negative Rate: [0.         0.025      0.09090909]
False Discovery Rate: [0.         0.07142857 0.03225806]
Overall Accuracy: [1.         0.96190476 0.96190476]
